In [1]:
# タイムスタンプ + MACアドレスのデータを
# MACアドレス, in時間, out時間に変換する

In [2]:
import pandas as pd

In [3]:
# csvファイルの読み込み
df = pd.read_csv('csv/20190123_1200_kanda.csv')

# df = pd.read_csv('csv/20190123_1200_kanda.csv', usecols = [2,4], skiprows = 1)
# df = pd.read_csv('csv/20190123_1200_kanda.csv', skiprows = 1)
# df = pd.read_csv('csv/20190123_1200_kanda.csv', skiprows = 2, header=None)
# df = pd.read_csv('csv/20190123_1200_kanda.csv', nrows = 50) # ちょっとだけ読み込む for debug

In [4]:
# 無効なデータが入っている1行目を削除
df = df.drop(0, axis=0)


In [5]:
# 日付をintに変換
df_int_cast = df.astype({'year': int, 'month': int, 'day': int, 'hour': int, 'minute': int, 'second': int})

In [6]:
# コピー
df_new = df_int_cast

In [7]:
# 時刻を秒に変換
days = 0 #df_int_cast['year'] * 365 + df_int_cast['month'] * 12 + df_int_cast['day']
hours = days * 24 + df_int_cast['hour']
minutes = hours * 60 + df_int_cast['minute']
df_new['elapsed_time'] = minutes * 60 + df_int_cast['second'] 

In [8]:
# 型を表示
print(df_new.dtypes)

year             int32
month            int32
day              int32
hour             int32
minute           int32
second           int32
address         object
elapsed_time     int32
dtype: object


In [9]:
# 時刻の列を削除
df_address_time = df_new.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1)

In [10]:
# Bleデータを管理するクラス
# addressが同じで前回計測されてからpersistence以内であればout_timeを更新する
# そうでなければFalseを返し、外部で新規BLEデバイスとしてリストに追加する
class BleLog:
    def __init__(self, address, time, persistence):
        self.address = address
        self.in_time = time 
        self.out_time = time
        self.persistence = persistence
        
    def update(self, address, time):
        
        # check address
        if self.address != address:
            return False
        
        # check elapsed time
        if time - self.out_time > self.persistence:
            return False
        
        # update out time
        self.out_time = time
        
        return True
    
    def print_data(self):
        print(self.address + " " + str(self.in_time) + " " + str(self.out_time) + " " + str(self.out_time - self.in_time))
        

In [11]:
# 現在のリストと比較する
def update_ble_data(address, time):
    for ble in ble_list:
        if ble.update(address, time):
            return True
    
    return False


In [12]:
# bleデータを表示する
def print_ble_data(data):
    for ble in data:
        ble.print_data()
    

In [13]:
# リストを初期化
ble_list = []

# csvからデータを読み込みbleリストと比較していく
# リストになければ新規にbleデバイスを追加
for index, row in df_address_time.iterrows():
    addr = row['address']
    time = row['elapsed_time']
    
    if not update_ble_data(addr, time):
        new_ble = BleLog(addr, time, 120)
        ble_list.append(new_ble)
        

In [14]:
len(ble_list)

736

In [15]:
# 結果を表示
print_ble_data(ble_list)

4C:54:9F:DD:0F:5F 43500 44149 649
55:07:CA:47:30:3C 43500 43510 10
4C:6C:63:7E:48:03 43500 43513 13
5D:29:48:14:0B:64 43500 43500 0
46:1A:BB:55:B7:A9 43500 43500 0
5D:31:81:CC:6A:A3 43500 43528 28
6E:1D:40:BE:E2:A9 43500 43500 0
4C:DB:F9:04:84:CA 43500 43500 0
76:39:C1:54:6E:28 43500 43533 33
78:30:D8:B7:AC:C5 43500 43576 76
6B:91:08:E6:43:1D 43500 43500 0
79:BF:1C:EF:F7:63 43500 43546 46
46:38:3B:E1:F3:71 43500 43523 23
42:7F:13:9B:AC:18 43500 43525 25
5F:50:72:16:2C:A3 43500 43577 77
65:AD:EC:49:B3:D2 43500 43650 150
44:FE:D9:98:54:E3 43500 43569 69
41:B1:5C:22:E4:6F 43501 43503 2
50:7C:2E:94:2F:09 43501 44154 653
78:8A:84:0A:3D:50 43501 43501 0
74:2D:2A:B1:2A:C5 43501 43516 15
61:FB:F4:BA:53:4E 43501 43505 4
7A:1E:39:F0:6F:00 43501 43585 84
4F:39:B3:0D:B5:53 43501 43545 44
70:41:30:0A:0D:20 43501 43551 50
4F:8F:2C:6C:BE:74 43501 43842 341
67:A1:CD:30:70:02 43501 43854 353
7F:79:54:A9:7F:A5 43501 43533 32
48:EB:1E:E3:40:81 43501 43597 96
7D:09:9A:E4:7C:E8 43501 43864 363
64:FA:82:99: